In [ ]:
import pandas as pd
df_train = pd.read_csv('./source/datasets/df_treino.csv')
df_test = pd.read_csv('./source/datasets/df_teste.csv')


In [144]:
print(df_train['Resultado'].value_counts())
print(df_test['Resultado'].value_counts())

0    5626
1     158
Name: Resultado, dtype: int64
0    1409
1      32
Name: Resultado, dtype: int64


In [145]:
y_train = df_train['Resultado'].to_numpy()
X_train = df_train.drop(['Resultado'], axis=1)#.to_numpy()

In [151]:
y_train_save = y_train
X_train_save = X_train

## 1) Balanceando as classes

Primeiramente, precisamos garantir representatividade de ao menos 1,5x da classe majoritária sobre a minoritária. Como temos consideravelmente mais instâncias de pessoas que não faleceram, vamos buscar utilizar diferentes técnicas tanto para selecionar instâncias da classe majoritária quanto para aumentar as instâncias da classe majoritária

Para isso, iremos utilizar a biblioteca imblearn

In [146]:
# !pip install imblearn

Primeiramente, aumentar a instância da classe minoritaria até atingir 20% da majoritária

In [147]:
from imblearn.over_sampling import SMOTE
import numpy as np

oversample = SMOTE(sampling_strategy=0.2, random_state=42)
print("Quantidade da classe minoritária antes da aumentação: ", np.unique(y_train, return_counts=True))

X_train, y_train = oversample.fit_resample(X_train, y_train)
print("Quantidade da classe minoritária após aumentação: ", np.unique(y_train, return_counts=True))

Quantidade da classe minoritária antes da aumentação:  (array([0, 1]), array([5626,  158]))
Quantidade da classe minoritária após aumentação:  (array([0, 1]), array([5626, 1125]))


Agora, vamos utilizar undersampling para reduzir a classe majoritária. 

In [148]:
from imblearn.under_sampling import NearMiss

nm = NearMiss(sampling_strategy={0:1687,1:1125})
X_train, y_train = nm.fit_resample(X_train, y_train)
print(X_train.shape)
print(y_train.shape)

(2812, 20)
(2812,)


In [150]:
import numpy as np
print(np.unique(y_train, return_counts=True))

(array([0, 1]), array([1687, 1125]))


## 2) Treinamento do modelo

A análise anterior foi usada apenas para demonstrar como devem ser balanceados os modelos. O balanceamento será realizado dentro dos folds de treinamento para cada iteração do k-fold, a fim de que a valicação reflita melhor os padrões que serão vistos nos conjuntos de teste